In [ ]:

from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain.chains import ConversationChain, RetrievalQA
from langchain.memory import ConversationBufferMemory
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.prompts import PromptTemplate
import secret

c:\LUMINAR\PROJECT\chatbot\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
#007bff

In [ ]:
gemini_model=ChatGoogleGenerativeAI(model='gemini-pro', google_api_key=secret.api_key)
memory=ConversationBufferMemory()
conversation=ConversationChain(llm=gemini_model, memory=memory)

C:\Users\hariv\AppData\Local\Temp\ipykernel_19904\3808084825.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory=ConversationBufferMemory()
C:\Users\hariv\AppData\Local\Temp\ipykernel_19904\3808084825.py:3: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  conversation=ConversationChain(llm=gemini_model, memory=memory)


In [3]:
def chatbot(prompt):
      response=conversation.predict(input=prompt)
      return response

In [5]:
loader=TextLoader(r"C:\LUMINAR\PROJECT\chatbot\sample.txt")
documents = loader.load()
text_splitter=RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=10)
text=text_splitter.split_documents(documents)

In [ ]:
embeddings=GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=secret.api_key)
vector_store=Chroma.from_documents(text, embeddings, persist_directory="./chroma_db")
vector_store.persist()

C:\Users\hariv\AppData\Local\Temp\ipykernel_19904\800809422.py:3: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vector_store.persist()


In [8]:
prompt_template = """
Use the following context to answer the question. If unsure, say you don't know.

Context:
{context}

Question:
{question}

Answer:
"""
prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

In [10]:
qa_chain = RetrievalQA.from_chain_type(
    llm=gemini_model,
    chain_type="stuff",
    retriever=vector_store.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

In [11]:
def rag_chat(user_input):
    response = qa_chain.run(user_input)
    return response

In [12]:
def chatbot():
    print("Choose mode: 1. Normal Chatting 2. RAG-Based")
    mode = input("Enter 1 or 2: ")
    if mode == "1":
        # Normal chat loop
        while True:
            user_input = input("You: ")
            if user_input.lower() in ["exit", "quit"]:
                break
            response = normal_chat(user_input)
            print(f"Bot: {response}")
    elif mode == "2":
        # RAG chat loop
        while True:
            user_input = input("You: ")
            if user_input.lower() in ["exit", "quit"]:
                break
            response = rag_chat(user_input)
            print(f"Bot: {response}")

In [ ]:
if __name__ == "__main__":
    chatbot()

Choose mode: 1. Normal Chatting 2. RAG-Based
